# An Example of how to generate a Matrix representation of a Hamiltonain from bit strings

Here I only consider one spin type 

In [1]:
import numpy as np
import pandas as pd

## First we define the function which generates the bit string states

In [2]:
#Creates all states with N fermions and S orbitals
def Generate_States(N,S):
    s = [i for i in range(0,N)]
    psi_l = []
    while s[0] < S-N:
        ### create the state and store it
        psi = 0
        for i in s:
            psi += 2**i
        psi_l.append(psi)
        ###
        ###Update the particle locations
        exit = 0
        i = len(s)-1
        while exit == 0:
            if s[i] < S-len(s)+i:
                s[i] += 1
                for j in range(i+1,len(s)):
                    s[j] = s[j-1]+1
                exit = 1
            else:
                i -= 1
        ###
    ###Create the finale state
    psi = 0
    for i in s:
        psi += 2**i
    psi_l.append(psi)
    ###
    return psi_l

In [3]:
states = Generate_States(2,4)
states

[3, 5, 9, 6, 10, 12]

### Function to print out the bit strings from integers

In [4]:
# A function to print out the binary number
def bi(num):
    bi = bin(num)
    out = ""
    for i in range(2,len(bi)):
        out = out + bi[i]
    return out

In [5]:
bi(states[0])

'11'

## Now define the action of the Hamiltonian on a bit string

I only consider hopping for on spin type

In [6]:
def bi_t(V,Su):
    Vn_l = []
    sign = 0
    for i in range(0,Su):
        #Spin up
        M = 2**i + 2**np.mod(i+1,Su)
        K = M & V
        L = K ^ M
        if L != 0 and L != M:
            Vn = V - K + L
            if i + 1 == np.mod(i+1,Su):
                sign = 1
            elif Su % 2 == 1:
                sign = 1
            else:
                sign = -1
            #print(i,':',bi(Vn))
            Vn_l.append([Vn,sign])
        #Spin down
        #Only doing spin up
    return Vn_l

In [7]:
V = states[4]
print(bi(V))
out = bi_t(V,4)
print('......')
for i in range(len(out)):
    print(bi(out[i][0]), out[i][1])

1010
......
1001 1
1100 1
110 1
11 -1


## Create the Matrix

We apply the bitwise Hamiltonian to the bit strings and collect the index of the output states using a dictionary

In [8]:
t = -1
S = 4
Q = len(states)
index_map = {states[i]:i for i in range(Q)}
H = np.array([[0 for i in range(Q)] for j in range(Q)])
for i in range(0,Q):
    psi_t = bi_t(states[i],S)
    for s in range(len(psi_t)):
        H[index_map[psi_t[s][0]],i] = psi_t[s][1]*t


        
H

array([[ 0, -1,  0,  0,  1,  0],
       [-1,  0, -1, -1,  0,  1],
       [ 0, -1,  0,  0, -1,  0],
       [ 0, -1,  0,  0, -1,  0],
       [ 1,  0, -1, -1,  0, -1],
       [ 0,  1,  0,  0, -1,  0]])

To do the full Hamiltonian I need to define bit string states for spin down.  The stratagy will be to create a second Generate_States function where all integers are shifted up.  

Actually, I could just make copies of the bi_t output since the up and down electrons should act the same. 

Note: there is a sign prblem with bi_t.  If there is an even number of electrons then going "around the world" should change the sign.

In [9]:
pwd

'/Users/stenger/Documents/Research/Hubbard_symmetries/VQE'

## Adding the spin down terms

In [10]:
I0 = np.identity(len(H))
Hf = np.kron(I0,H)+np.kron(H,I0)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.DataFrame(Hf)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,0.0,-1.0,0.0,0.0,1.0,0.0,-1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.0,0.0,-1.0,-1.0,0.0,1.0,-0.0,-1.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-0.0,-1.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-0.0,0.0,-1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,0.0,-0.0,-0.0,-1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-1.0,-0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0,0.0,-1.0,-0.0,0.0,0.0,0.0,0.0,-1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,-0.0,-1.0,-0.0,-0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,1.0,-0.0,-1.0,-0.0,-0.0,0.0,0.0,-0.0,-1.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.0,-0.0,-1.0,0.0,-0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-0.0,-1.0,0.0,-0.0,0.0,0.0,-0.0,-1.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0.0,-0.0,0.0,-1.0,-0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-0.0,0.0,-1.0,-0.0,0.0,0.0,-0.0,0.0,-1.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Adding the interactions

Need to either create all of the states or define a function apply U directly to the matrix.

In [11]:
states_full = []
for i in range(0,len(states)):
    for j in range(0,len(states)):
        states_full.append(states[i]*2**S+states[j])

for psi in states_full:
    print(bi(psi))

110011
110101
111001
110110
111010
111100
1010011
1010101
1011001
1010110
1011010
1011100
10010011
10010101
10011001
10010110
10011010
10011100
1100011
1100101
1101001
1100110
1101010
1101100
10100011
10100101
10101001
10100110
10101010
10101100
11000011
11000101
11001001
11000110
11001010
11001100


In [12]:
def bi_u(V,S):
    Su = int(S/2)
    Vn_l = []
    for i in range(0,Su):
        M = 2**i + 2**(i+Su)
        K = M & V
        if K == M:
            Vn = V
            Vn_l.append(Vn)
    return Vn_l

In [13]:
bi_u(states_full[0],8)

[51, 51]

In [44]:
u = 9.0
S = 4
Qf = len(states_full)
index_map = {states_full[i]:i for i in range(Qf)}
Hu = np.array([[0.0 for i in range(Qf)] for j in range(Qf)])
for i in range(0,Qf):
    psi_t = bi_u(states_full[i],2*S)
    for s in range(len(psi_t)):
        Hu[index_map[psi_t[s]],i] += u


In [45]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.DataFrame(Hu)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
H_tu = Hf + Hu
pd.DataFrame(H_tu)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,18.0,-1.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.0,9.0,-1.0,-1.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,-1.0,9.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,-1.0,0.0,9.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,-1.0,-1.0,9.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-1.0,0.0,0.0,0.0,0.0,0.0,9.0,-1.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,18.0,-1.0,-1.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,9.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,9.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [47]:
np.save('H_mu_'+str(u)+"_t_"+str(t),H_tu)

## Convert each block into a sum of Puali terms

Let us start with an example using the first block.  Each block is S choose N big.

In [18]:
from qiskit.opflow import (I, X, Y, Z)
from qiskit.opflow.primitive_ops import MatrixOp

The issue is that the size of the matrix does not generally match the size of a qubit space.  We have large numbers to the diagonal to get the size to match and to prevent the added sited from having the lowest eigenvalues.

In [19]:
def complete(b11):
    size = 2**np.ceil(np.log2(len(b11)))
    for v in b11:
        while len(v) < size:
            v.append(0.0)
    while len(b11) < size:
        i = len(b11)
        vnew = [0.0 for i in range(0,8)]
        vnew[i] = 1000
        b11.append(vnew)
    return b11

complete(b11)

NameError: name 'b11' is not defined

In [95]:
import math
N = 4
S = 2
bL = math.comb(4,2)

blocks = {}
for bi in range(bL):
    for bj in range(bi,bL):
        bij = [[H_tu[i + bL*bi][j + bL*bj] for j in range(0,bL)] for i in range(0,bL)]
        if np.amax(np.abs(bij)) > 10**(-5):
            bij = complete(bij)
            bij_pauli = MatrixOp(bij).to_pauli_op()
            blocks[str(bi) + ',' + str(bj)] = bij_pauli
        



In [96]:
blocks.keys()

dict_keys(['0,0', '0,1', '0,4', '1,1', '1,2', '1,3', '1,5', '2,2', '2,4', '3,3', '3,4', '4,4', '4,5', '5,5'])

In [99]:
pd.DataFrame(blocks['0,0'].to_matrix())

,0,1,2,3,4,5,6,7
0,2.0+0.0j,-1.0+0.0j,0.0+0.0j,0.0+0.0j,1.0+0.0j,0.0+0.0j,0.00+0.00j,0.00+0.00j
1,-1.0+0.0j,1.0+0.0j,-1.0+0.0j,-1.0+0.0j,0.0+0.0j,1.0+0.0j,0.00+0.00j,0.00+0.00j
2,0.0+0.0j,-1.0+0.0j,1.0+0.0j,0.0+0.0j,-1.0+0.0j,0.0+0.0j,0.00+0.00j,0.00+0.00j
3,0.0+0.0j,-1.0+0.0j,0.0+0.0j,1.0+0.0j,-1.0+0.0j,0.0+0.0j,0.00+0.00j,0.00+0.00j
4,1.0+0.0j,0.0+0.0j,-1.0+0.0j,-1.0+0.0j,1.0+0.0j,-1.0+0.0j,0.00+0.00j,0.00+0.00j
5,0.0+0.0j,1.0+0.0j,0.0+0.0j,0.0+0.0j,-1.0+0.0j,0.0+0.0j,0.00+0.00j,0.00+0.00j
6,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,1000.0+0.0j,0.00+0.00j
7,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.00+0.00j,1000.0+0.0j


In [98]:
bi = 0
bj = 0
[[H_tu[i + bL*bi][j + bL*bj] for j in range(0,bL)] for i in range(0,bL)]

[[2.0, -1.0, 0.0, 0.0, 1.0, 0.0],
 [-1.0, 1.0, -1.0, -1.0, 0.0, 1.0],
 [0.0, -1.0, 1.0, 0.0, -1.0, 0.0],
 [0.0, -1.0, 0.0, 1.0, -1.0, 0.0],
 [1.0, 0.0, -1.0, -1.0, 1.0, -1.0],
 [0.0, 1.0, 0.0, 0.0, -1.0, 0.0]]